# 12770 - comfo

## Energy & Carbon + Decision Tree

In [ ]:
from datetime import datetime
######################## ENERGY CONSUMPTION ########################

# we put literature values for *kw* consumption here to multiply by the hour used.
appliance_power = {
    "AC": 1.2,
    "fan": 0.06,
    "heater": 1.5,
    "space_heater": 0.75,
    "dehumidifier": 0.3,
    "window": 0.0
}

######################## CARBON EMISSION ########################

# pittsburgh grid emission factor (CO₂e per kWh) from eGRID2023 results. This should change based on the grid mix (ZIP code input by the user)
emissions =  {
    "pittsburgh": 0.33
    }

######################## RESULT / NETTO ########################

def get_emission_factor(city):
    # this function would ideally fetch the emission factor based on the city / ZIP code
    # for now, we will use a hardcoded value for Pittsburgh
    return emissions.get(city.lower(), 0.33)  # default to Pittsburgh if not found



def estimate_savings(replacement_action, baseline_action, duration_hours, city):
    """Calculate energy and carbon savings when switching actions"""
    if replacement_action not in appliance_power or baseline_action not in appliance_power:
        return None, None
    
    emission_factor = get_emission_factor(city)

    baseline_energy = appliance_power[baseline_action] * duration_hours
    replacement_energy = appliance_power[replacement_action] * duration_hours

    energy_saved = baseline_energy - replacement_energy
    carbon_saved = energy_saved * emission_factor

    return round(energy_saved, 2), round(carbon_saved, 2)


######################## DECISION TREE IMPLEMENTATION ########################

def get_season(date_str):
    date = datetime.strptime(date_str, "%Y-%m-%d")
    summer_start = datetime(date.year, 4, 1)
    summer_end = datetime(date.year, 10, 31)
    return "summer" if summer_start <= date <= summer_end else "winter"


def get_tau_category(tau_minutes):
    if tau_minutes < 780:
        return "fast"
    elif 780 <= tau_minutes < 2820:
        return "medium"
    else:
        return "slow"

suggestions = {
        ("summer", "hot", "fast"): {
            "Energy saving": [
                "Open windows for cross-ventilation if the outdoor temp is lower.",
                "Use ceiling or box fans instead of AC.",
                "Close blinds to reduce solar heat gain."
            ],
            "Comfort": [
                "Lower the thermostat by 1-2°C for a quick fix.",
                "Turn on a fan to circulate air more effectively.",
                "Place a bowl of ice in front of a fan for evaporative cooling."
            ]
        },
        ("summer", "hot", "medium"): {
            "Energy saving": [
                "Open windows only if the external temperature is lower than inside.",
                "Use a dehumidifier instead of AC.",
                "Close blinds or curtains in sun-exposed areas."
            ],
            "Comfort": [
                "Adjust thermostat by 1-2°C but expect it to take 30-45 minutes.",
                "Use oscillating fans to circulate air.",
                "Stay hydrated and apply cold compresses."
            ]
        },
        ("summer", "hot", "slow"): {
            "Energy saving": [
                "Pre-cool the room early in the morning or late evening.",
                "Use blackout curtains or shades.",
                "Use fans to help distribute cooler air."
            ],
            "Comfort": [
                "Lower the thermostat knowing it will take 1-2 hours to take effect.",
                "Use portable fans and cooling devices.",
                "Stay hydrated and avoid heat-generating activities."
            ]
        },
        ("summer", "cold", "fast"): {
            "Energy saving": [
                "Ensure AC isn’t overcooling the space.",
                "Use fans to circulate air without lowering the thermostat further."
            ],
            "Comfort": [
                "Adjust thermostat up by 1-2°C.",
                "Close windows if cool air is leaking in."
            ]
        },
        ("summer", "cold", "medium"): {
            "Energy saving": [
                "Reduce cooling system use and rely on fans.",
                "Set thermostat to a slightly higher temperature."
            ],
            "Comfort": [
                "Turn the thermostat up gradually and check after 30-45 minutes.",
                "Avoid cooling too aggressively."
            ]
        },
        ("summer", "cold", "slow"): {
            "Energy saving": [
                "Keep cooling moderate to prevent excessive use.",
                "Use fans for local cooling."
            ],
            "Comfort": [
                "Be patient; avoid expecting rapid cooling.",
                "Close off unneeded spaces to retain cool air."
            ]
        },
        ("winter", "hot", "fast"): {
            "Energy saving": [
                "Turn down the thermostat and allow natural cooling.",
                "Open windows for quick ventilation."
            ],
            "Comfort": [
                "Reduce thermostat setting by 1-2°C.",
                "Check if heating appliances are adding unnecessary heat.",
                "Increase ventilation to balance the temperature."
            ]
        },
        ("winter", "hot", "medium"): {
            "Energy saving": [
                "Lower the thermostat gradually by 1°C.",
                "Ventilate briefly by opening windows."
            ],
            "Comfort": [
                "Adjust the thermostat by 1-2°C.",
                "Keep interior doors closed.",
                "Check for unnecessary heat gains."
            ]
        },
        ("winter", "hot", "slow"): {
            "Energy saving": [
                "Avoid large temperature swings.",
                "Maintain a consistent lower setpoint.",
                "Slighly open a window."
            ],
            "Comfort": [
                "Remove extra layers or blankets.",
                "Allow for drafts and remove thick curtains or blinds."
            ]
        },
        ("winter", "cold", "fast"): {
            "Energy saving": [
                "Use natural sunlight by opening blinds.",
                "Close off unused rooms to retain warmth."
            ],
            "Comfort": [
                "Adjust thermostat by 1-2°C for immediate heating.",
                "Layer up with warm clothes and blankets."
            ]
        },
        ("winter", "cold", "medium"): {
            "Energy saving": [
                "Set thermostat reasonably and allow time to heat.",
                "Close blinds and curtains at night."
            ],
            "Comfort": [
                "Use heated blankets or space heaters while warming.",
                "Wear warm clothing."
            ]
        },
        ("winter", "cold", "slow"): {
            "Energy saving": [
                "Use thermal curtains and insulate windows.",
                "Keep interior doors closed."
            ],
            "Comfort": [
                "Use warm clothing or heated blankets.",
                "Use space heaters strategically."
            ]
        },
    }

######################## MAIN ########################

def comfo():
    print("Welcome to Comfo!")
    city = input("Enter your city (e.g. Pittsburgh): ").strip()
    date_str = input("Enter today's date (YYYY-MM-DD): ")
    discomfort = input("Is it too hot or too cold? (hot/cold): ").strip().lower()
    tau = int(input("Enter your tau value in minutes (e.g., 450): "))

    season = get_season(date_str)
    tau_category = get_tau_category(tau)

    print(f"\nSeason: {season.capitalize()}, Discomfort: {discomfort}, Tau: {tau_category.capitalize()}\n")

    
    key = (season, discomfort, tau_category)

    if key in suggestions:
        for category, tips in suggestions[key].items():
            print(f"{category} Tips:")
            for replacement, baseline, description in tips:
                if replacement in appliance_power and baseline in appliance_power:
                    duration_
                    energy, carbon = estimate_savings(replacement, baseline, duration_hours, city)
                    print(f" - {description} (Save {energy:.2f} kWh, {carbon:.2f} kg CO₂)")
                else:
                    print(f" - {description} (Low carbon action)")
            print()
    else:
        print("Sorry, no suggestions available for that combination.")

if __name__ == "__main__":
    comfo()


# Test case: using a fan instead of AC for 2 hours
result = estimate_savings("fan", "AC", 2, 'pittsburgh')
print(f"Energy Saved: {result[0]} kWh")
print(f"Carbon Saved: {result[1]} kg CO2")


Energy Saved: 2.28 kWh
Carbon Saved: 0.75 kg CO2
Welcome to Comfo!
